<a href="https://colab.research.google.com/github/avimec13/project1/blob/master/CNN_Time_Series_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Single Step Convolutional Nueral Network

# CNN Time Series Forecasting

In [41]:
#Importing Libraries
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [43]:
# split a univariate sequence into samples
def split_sequence(sequence, steps):
  X, y = list(), list()
  for start in range(len(sequence)):
    # define the end index of the sequence
    end_index = start + steps
    # to check if end_index stays in the allowable limit
    if end_index > len(sequence)-1:
      break
    # extract input and output parts of the sequence
    sequence_x, sequence_y = sequence[start : end_index], sequence[end_index]
    X.append(sequence_x)
    y.append(sequence_y)
  return array(X), array(y)

In [44]:
raw_sequence = [100, 110, 120, 130, 140, 150 ,160, 170, 180 ,190]

# choose a number of time steps
steps = 3

# split into samples
X, y = split_sequence(raw_sequence, steps)

print("**************************************************************")
print("The input raw sequence on which we will test our CNN model")
print(raw_sequence)
print("**************************************************************")
print("The features (X) ,lagged values of time series, for our CNN model")
print(X)
print("**************************************************************")
print("The respective observed values (y) of the sequence for training in CNN model")
print(y)
print("**************************************************************")

**************************************************************
The input raw sequence on which we will test our CNN model
[100, 110, 120, 130, 140, 150, 160, 170, 180, 190]
**************************************************************
The features (X) ,lagged values of time series, for our CNN model
[[100 110 120]
 [110 120 130]
 [120 130 140]
 [130 140 150]
 [140 150 160]
 [150 160 170]
 [160 170 180]]
**************************************************************
The respective observed values (y) of the sequence for training in CNN model
[130 140 150 160 170 180 190]
**************************************************************


In [45]:
# reshape 2D input data to 3D input data
features = 1
X = X.reshape((X.shape[0], X.shape[1], features))

print("After reshaping , the shape of input X")
print(X.shape)
print("**************************************************************")
print("**************************************************************")
print("The final form of input feature matrix X")
print(X)
print("**************************************************************")
print("**************************************************************")
print("Feature matrix X is ready for input to CNN model.We have used feature engineering" 
    +"to convert a sequence to matrix with image matrix shape to find patterns in sequence")
print("**************************************************************")

After reshaping , the shape of input X
(7, 3, 1)
**************************************************************
**************************************************************
The final form of input feature matrix X
[[[100]
  [110]
  [120]]

 [[110]
  [120]
  [130]]

 [[120]
  [130]
  [140]]

 [[130]
  [140]
  [150]]

 [[140]
  [150]
  [160]]

 [[150]
  [160]
  [170]]

 [[160]
  [170]
  [180]]]
**************************************************************
**************************************************************
Feature matrix X is ready for input to CNN model.We have used feature engineeringto convert a sequence to matrix with image matrix shape to find patterns in sequence
**************************************************************


In [46]:
# Defining model Architecture
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(steps,
features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [47]:
# fit model
model.fit(X, y, epochs=10000, verbose=0)

In [49]:
#Testing the Model
x_input = array([ 170, 180, 190 ])
x_input = x_input.reshape((1, steps, features))
y_pred = model.predict(x_input, verbose=0)
print("**************************************************************")
print(f"The next predicted value for the sequence {raw_sequence} is")
print(y_pred)
print("**************************************************************")



**************************************************************
The next predicted value for the sequence [100, 110, 120, 130, 140, 150, 160, 170, 180, 190] is
[[200.31905]]
**************************************************************


Multi-step Convolutional 1D Model 

In [50]:
# univariate multi-step vector-output 1d cnn example
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out
    # check if we are beyond the sequence
    if out_end_ix > len(sequence):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps_in,
n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(105, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=7000, verbose=0)

# demonstrate prediction
x_input = array([ 70,80, 90 ])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)



[[104.11041 116.68879]]
